In [2]:
import pandas as pd

In [12]:
akas = pd.read_csv('./data/imdb/title_akas.tsv', sep = "\t").dropna()
ratings = pd.read_csv('./data/imdb/title_ratings.tsv', sep = "\t")

<ipython-input-12-32fb714f1fa2>:1: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  akas = pd.read_csv('./data/imdb/title_akas.tsv', sep = "\t").dropna()


In [13]:
akas.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [6]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1922
1,tt0000002,5.8,259
2,tt0000003,6.5,1734
3,tt0000004,5.6,174
4,tt0000005,6.2,2546


In [14]:
akas.title = akas.title.apply(lambda x: x.lower().strip())

In [17]:
pre = akas.shape[0]

In [18]:
akas.drop_duplicates(subset = 'title', inplace = True)

In [20]:
pre - akas.shape[0]

29735100

,titleId
title,
карменсіта,tt0000001
carmencita,tt0000001
carmencita - spanyol tánc,tt0000001
καρμενσίτα,tt0000001
карменсита,tt0000001


In [8]:
movie_meta_columns = ['movie_id',
                    'freebase_movie_id',
                    'movie_name',
                    'release_date',
                    'box_office_revenue',
                    'runtime',
                    'languages',
                    'countries',
                    'genres']

movie_metadata = pd.read_csv('./data/movie.metadata.tsv', sep='\t', header=None, names=movie_meta_columns)
movie_metadata.movie_name = movie_metadata.movie_name.apply(lambda x: x.lower().strip())
movie_metadata = movie_metadata[["movie_id", "movie_name"]].set_index("movie_name")

movie_metadata.head()

,movie_id
movie_name,
ghosts of mars,975900
getting away with murder: the jonbenét ramsey mystery,3196793
brun bitter,28463795
white of the eye,9363483
a woman in flames,261236


In [22]:
merge = pd.merge(movie_metadata, akas, left_index= True, right_index = True)
merge.head()

,movie_id,titleId
!women art revolution,29988427,tt1699720
#1 cheerleader camp,30332673,tt1637976
$,4213160,tt0068152
$9.99,20624798,tt0439051
'68,2250713,tt0094587


In [23]:
merge = merge.drop_duplicates(subset = 'movie_id')

In [24]:
freebase_movies_imdb_score = pd.merge(merge.set_index("titleId"), ratings.set_index("tconst"), left_index=True, right_index=True)
freebase_movies_imdb_score = freebase_movies_imdb_score.reset_index().drop("index", axis = 1).set_index("movie_id")

In [25]:
freebase_movies_imdb_score

,averageRating,numVotes
movie_id,,
7669437,5.7,1922
9996498,5.8,259
6431647,6.5,1734
9999926,5.6,174
73372,6.2,2546
...,...,...
24319366,6.2,78
18475018,5.8,6
7316179,6.0,6


In [28]:
print(f"{movie_metadata.shape[0]-freebase_movies_imdb_score.shape[0]} \
freebase titles lost in merge (\
{100 * (movie_metadata.shape[0]-freebase_movies_imdb_score.shape[0])/movie_metadata.shape[0]:.3f})%")

17239 freebase titles lost in merge (21.090)%


In [55]:
saved = freebase_movies_imdb_score.copy()